# Update datasets in GC which have newer versions available
- not sufficiently tested - BE CAREFUL

In [1]:
import numpy as np
import pandas as pd
import os
import gcsfs
import xarray as xr
from functools import partial
from IPython.display import display
from glob import glob
import warnings
import datetime

### Local modules

In [2]:
from request import requests, set_request_id
from search import search_new, esgf_search_sites
from netcdf import get_ncfiles, concatenate
from identify import needed_newversion, get_version
from response import response, dict_to_dfcat, get_details

In [3]:
def getFolderSize(p):
    prepend = partial(os.path.join, p)
    return sum([(os.path.getsize(f) if os.path.isfile(f) else getFolderSize(f)) for f in map(prepend, os.listdir(p))])

### Initialization

In [4]:
# Where to write NEW zarr stores:
zarr_local = '/h68/naomi/zarr-minimal'

#zarr_local = '/d1/naomi/cmip6-zarrs'  # usually matches location in nb1-DataRequests

#local_storage = True
local_storage = False

fs = gcsfs.GCSFileSystem(token='anon', access='read_only')

### Choose basic configuration parameters

In [5]:
dtype = esgf_search_sites()

local_node=False

print('possible ESGF API search nodes: ',list(dtype.keys()))

ESGF_site = dtype['llnl']#;local_node = True

#ESGF_site = dtype['gfdl']
#ESGF_site = dtype['ipsl']
#ESGF_site = dtype['ceda']

# List sites to skip for aquiring new netcdf files: broken or slow sites
skip_sites = []#'esg.lasg.ac.cn','esgf-data2.diasjp.net','esgf-cnr.hpc.cineca.it'] #['dist.nmlab.snu.ac.kr']

possible ESGF API search nodes:  ['llnl', 'ipsl', 'nci', 'ceda', 'jpl', 'gfdl', 'dkrz']


### Get prior Google Sheet requests

In [6]:
df_prior = pd.read_csv('csv/requests.csv')
#df_prior
df_prior.keys()

Index(['Timestamp', 'E-mail', 'response status', 'members', 'experiments', 'models', 'variables', 'table', 'requester', 'science', 'comments'], dtype='object')

### Get new Google Sheet requests
- by default, only the new rows from the sheet are considered
- specifying a list of rows or emails will add older entries 

In [7]:
rows = []   
emails = []

# modify here:

rows = [0]

#emails = ['neil.swart@canada.ca']
df_request_new, dtrouble = requests(df_prior,rows=rows,emails=emails)

request_id = set_request_id()

# Check for mal-formed requests (non-existent variables, etc)
if len(dtrouble)>=1:
    print(dtrouble)

df_request_new

,Timestamp,E-mail,response status,members,experiments,models,variables,table,requester,science,comments
0,9/9/2019 14:13:56,rpa@ldeo.columbia.edu,frequent,[All],"[ssp126, ssp245, ssp370, ssp585, ssp119, ssp43...",[All],"[ts, psl, hfls, ps, sfcWind, uas, vas, tas]",Amon,Ryan,Basics,NaN
183,2020-06-13,naomi@ldeo.columbia.edu,frequent,[All],[historical],[All],"[pr, prc, ts, ps, tas]",Amon,naomi,Basics,NaN


In [8]:
# choose a new request to process:
timestamps = df_request_new.Timestamp.unique()
print(timestamps)
#df_request_new = df_request_new[df_request_new.Timestamp == timestamps[-1]]
df_request_new = df_request_new[df_request_new.requester == 'naomi']

df_request_new

['9/9/2019 14:13:56' '2020-06-13']


,Timestamp,E-mail,response status,members,experiments,models,variables,table,requester,science,comments
183,2020-06-13,naomi@ldeo.columbia.edu,frequent,[All],[historical],[All],"[pr, prc, ts, ps, tas]",Amon,naomi,Basics,NaN


### Search ESGF for the availability of requested data

In [9]:
print(ESGF_site)
df_ESGF = search_new(ESGF_site,df_request_new,local_node=local_node)

https://esgf-node.llnl.gov/esg-search/search

naomi@ldeo.columbia.edu
Amon pr ['All'] historical
Amon prc ['All'] historical
Amon ts ['All'] historical
Amon ps ['All'] historical
Amon tas ['All'] historical


### Get the master list of existing zarr stores
- df_master includes all curated stores
- df_avail includes all stores, even those with known ES-DOC issues 

In [10]:
import qgrid
df_avail = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv')
len(df_avail),len(df_ESGF)
#qgrid.show_grid(df_ESGF)

/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(274251, 40210)

### Check the new requests:
- already exists in df_avail (what we have) - is version the same? if so, then not needed.
- exists in df_ESGF (what is available) - if not available, then not needed|

In [11]:
df_needed = needed_newversion(df_avail, df_request_new, df_ESGF)

if len(df_needed) > 0:
    num_stores = df_needed.zstore.nunique() 
    print(f'needed: nfiles={len(df_needed)}, nstores={num_stores}')
    #print(df_needed.zstore.unique())
else:
    print('no new data available')
    exit

['v20190306' 'v20190429']
new version available gs://cmip6/CMIP/E3SM-Project/E3SM-1-0/historical/r1i1p1f1/Amon/pr/gr/ new:  20190913 old:  ambiguous
new version available gs://cmip6/CMIP/E3SM-Project/E3SM-1-0/historical/r4i1p1f1/Amon/pr/gr/ new:  20190916 old:  ambiguous
new version available gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r1i1p1f1/Amon/pr/gr/ new:  20200225 old:  20190605
new version available gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r2i1p1f1/Amon/pr/gr/ new:  20200226 old:  20190911
new version available gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r3i1p1f1/Amon/pr/gr/ new:  20200226 old:  20190911
new version available gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r4i1p1f1/Amon/pr/gr/ new:  20200425 old:  20190728
new version available gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3/historical/r1i1p1f1/Amon/pr/gr/ new:  20200310 old:  20190926
new version available gs://cmip6/CMIP/EC-Earth-Consortium/EC-E

In [12]:
assert False 

AssertionError: 

### Start logging the progress and exceptions

In [ ]:
cat_file = 'csv/cmip6_'+request_id+'.csv'
log_file = 'txt/request_'+request_id+'.log'
print(log_file)

In [ ]:
# open and close for each write in case of kernel interrupt
def write_log(file,str,verbose=True):
    f = open(file,'a')
    if verbose:
        print(str)
    f.write(str+'\n')
    f.close()
    return

In [ ]:
df_needed['member'] = [int(s.split('r')[-1].split('i')[0]) for s in df_needed['member_id']]
df_needed = df_needed.sort_values(by=['source_id'])
df_needed = df_needed.sort_values(by=['member'])
#df_needed

### The real work is done in this next loop 
- could be done in parallel except for the writing to the log file

In [ ]:
# reload the catalog
df_GCS = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv', dtype='unicode')

date = str(datetime.datetime.now().strftime("%Y%m%d"))
os.system('cp /home/naomi/cmip6-zarr/csv/pangeo-cmip6-noQC.csv /home/naomi/cmip6-zarr/csv/pangeo-cmip6-'+date+'-noQC.csv')

if local_storage:
    df_loc = pd.read_csv('shelf-new/local.csv', dtype='unicode')

# refresh the gcsfs
fs = gcsfs.GCSFileSystem(token='anon', access='read_only',cache_timeout=-1)

new_zarrs = df_needed.zstore.unique()

verbose = True

zbdirs = []
for i in range(1,84):
    zbdirs += ['/h'+str(i)]

for item,zarr in enumerate(new_zarrs):
    #if item >0:
    #    continue
    
    zbdir  = zarr_local  + zarr
    zbtemp = 'zbtemp' + zarr
    
    write_log(log_file,f"\n>>{item+1}/{num_stores}:<< local file: {zbdir}",verbose=verbose)
    
    # is zarr already in cloud?  Unreliable
    gsurl = 'gs://cmip6' + zarr
    
    try:
        dataset_id, version_id = get_version(gsurl)
    except:
        version_id = 'unknown'
    
    cstore = df_GCS[df_GCS.zstore == gsurl]
    if len(cstore) > 0:
       print(cstore.zstore.values,cstore.version.values)
       print('store in cloud catalog')  # or on a shelf drive which is already in cloud
       #continue
        
    new_version = df_needed[df_needed.zstore==zarr].version.unique()
    
    if new_version[0][1:] == version_id:
        print('new version already uploaded',version_id)
        continue
        
    # CHECK IN shelf-new/local.csv
    if local_storage:
        df1 = df_loc[df_loc['zstore'].str.contains(zarr[:-1])]

    gfiles,troubles,codes,okay = get_ncfiles(zarr,df_needed,skip_sites)
    
    write_log(log_file,troubles,verbose=verbose)
    
    if okay == False:
        continue

    if len(gfiles) == 0: 
        write_log(log_file,'no files available',verbose=verbose)
        continue
    
    variable_id = zarr.split('/')[-3]
    for gfile in gfiles:   # changes file sizes!!
        command = '/usr/bin/ncatted -h -O -a missing_value\,'+variable_id+',d,, '+gfile
        os.system(command)

    # concatenate in time with mfdataset
    #print(gfiles)
    gfiles = sorted(gfiles)
    status, ds, dstr = concatenate(zarr,gfiles,codes)  

    if status == 'failure':
        write_log(log_file,status+dstr,verbose=verbose)
        continue
    else:
        write_log(log_file,dstr)

    # 0. save new zstore
    try:
        del ds[variable_id].encoding['missing_value']
        ds.to_zarr(zbtemp, consolidated=True, mode='w')
    except:
        ds.to_zarr(zbtemp, consolidated=True, mode='w')
        
    if not os.path.isfile(zbtemp+'/.zmetadata'):
        write_log(log_file,'to_zarr failure: ',verbose=verbose)
        continue
     
    # 1. delete old version in GC
    command = '/usr/bin/gsutil -m rm -r '+ gsurl[:-1]
    print(command)
    os.system(command) 
    
    # 2. delete entry in ncsv/GC_files_{activity_id}-{institution_id}.csv
    activity_id = cstore.activity_id.values[0]
    institution_id = cstore.institution_id.values[0]
    
    file = f'/home/naomi/cmip6-zarr/ncsv/GC_files_{activity_id}-{institution_id}.csv'
    #print('modifying ',file)
    with open(file, "r") as f:
        lines = f.readlines()
    with open(file, "w") as f:
        for line in lines:
            if line.strip("\n") != gsurl + ".zmetadata":
                f.write(line)
        
    if local_storage:
        # 3. delete old local copy(ies)
        for zloc in df1.zstore.values:
            if os.path.exists(zloc):
                command = '/bin/rm -rf '+ zloc
                print(command)
                os.system(command)
            else:
                print('not active: ',zloc)
    
        # 4. delete entry(ies) in shelf-new/h*.csv
        for zloc in df1.zstore.values:
            file = 'shelf-new/' + zloc.split('/')[1] + '.csv'
            print(file)
            writeable = os.access(file, os.W_OK)
            if not writeable:
                command = "chmod u+w " + file
                #print(command)
                os.system(command)

            dfff = pd.read_csv(file)
            dff = dfff[dfff.zstore != zloc]
            dff.to_csv(file, mode='w+', index=False)

            if not writeable:
                command = "chmod u-w " + file
                #print(command)
                os.system(command)

        # 5. copy zbtemp to zbdir
        grid_label = zbdir[:-1].split('/')[-1]
        command = 'mkdir -p ' + zbdir.split(grid_label)[0]
        os.system(command) 
        command = '/bin/cp -r '+ zbtemp[:-1] + ' ' + zbdir[:-1]
        write_log(log_file,command,verbose=verbose)
        os.system(command) 
    
    # 6. upload to cloud
    command = '/usr/bin/gsutil -m cp -r '+ zbtemp[:-1] + ' ' + gsurl[:-1]
    write_log(log_file,command,verbose=verbose)
    os.system(command) 
        
    size_remote = fs.du(gsurl)
    size_local = getFolderSize(zbdir)
    assert (size_remote - size_local) < 100

    try:
        ds = xr.open_zarr(fs.get_mapper(gsurl), consolidated=True)
        write_log(log_file,f'successfully saved as {gsurl}')
        for gfile in gfiles:
            os.system('rm -f '+ gfile)
    except:
        write_log(log_file,'store did not get saved to GCS properly')
